In [2]:
!pip install jaconv

  Preparing metadata (setup.py) ... done
  Created wheel for jaconv: filename=jaconv-0.4.0-py3-none-any.whl size=18228 sha256=7de400694075b0cd652ab38dcc95a8dca15644aa28f300aee2d8168ab9e0bc53
  Stored in directory: /root/.cache/pip/wheels/c9/97/53/1f827ebf916b899520ce663227497206525c432be72cc29265
Successfully built jaconv


In [3]:
# ライブラリのインポート
import pandas as pd
from google.colab import files
import io
import jaconv
import re

In [4]:
# 料理データセットの読み込み
uploaded = files.upload()  # ファイル選択ダイアログを出す
file_name = list(uploaded.keys())[0]
xls = pd.ExcelFile(io.BytesIO(uploaded[file_name]))  # Excelファイルのシート名を確認
print(xls.sheet_names)  # Excelのシート名の確認

df_recipes_ori = pd.read_excel(xls, sheet_name='recipes')
df_prebiotics_ori = pd.read_excel(xls, sheet_name='prebiotics')
df_probiotics_ori = pd.read_excel(xls, sheet_name='probiotics')

Saving 料理名と食材・栄養素のデータセット.xlsx to 料理名と食材・栄養素のデータセット.xlsx
['recipes', 'prebiotics', 'probiotics']


In [24]:
df_recipes = df_recipes_ori.copy()
df_prebiotics = df_prebiotics_ori.copy()
df_probiotics = df_probiotics_ori.copy()

In [25]:
# 列をすべて表示
pd.set_option('display.max_columns', None)

# 行をすべて表示
pd.set_option('display.max_rows', None)
df_recipes.head()

,料理の種類,料理名,エネルギー[キロカロリー],たんぱく質[グラム],脂質[グラム],炭水化物[グラム],食塩相当量[グラム],野菜量[グラム],材料1＿材料名,材料1＿重量[グラム],材料2＿材料名,材料2＿重量[グラム],材料3＿材料名,材料3＿重量[グラム],材料4＿材料名,材料4＿重量[グラム],材料5＿材料名,材料5＿重量[グラム],材料6＿材料名,材料6＿重量[グラム],材料7＿材料名,材料7＿重量[グラム],材料8＿材料名,材料8＿重量[グラム],材料9＿材料名,材料9＿重量[グラム],材料10＿材料名,材料10＿重量[グラム],材料11＿材料名,材料11＿重量[グラム],材料12＿材料名,材料12＿重量[グラム],材料13＿材料名,材料13＿重量[グラム],材料14＿材料名,材料14＿重量[グラム],材料15＿材料名,材料15＿重量[グラム],材料16＿材料名,材料16＿重量[グラム],材料17＿材料名,材料17＿重量[グラム],材料18＿材料名,材料18＿重量[グラム],材料19＿材料名,材料19＿重量[グラム],材料20＿材料名,材料20＿重量[グラム]
0,ごはん,ごはん（中茶碗1杯）,252,3.8,0.5,55.7,0.0,0.0,めし・精白米（水稲）,150.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ごはん,麦ごはん,248,4.3,0.7,54.0,0.0,0.0,米・精白米（水稲）,60.0,大麦・押麦,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ごはん,きのこたっぷり混ぜご飯,307,6.6,2.4,61.4,1.0,23.0,米・精白米（水稲）,70.0,えのきたけ＿生,7.0,しめじ・ぶなしめじ＿生,7.0,まいたけ＿生,7.0,油揚げ,5.0,かつお・昆布だし,90.0,こいくちしょうゆ,6.0,みりん風調味料,9.0,清酒・上撰,7.5,糸みつば・葉＿生,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ごはん,五目炊き込みご飯,311,11.4,2.7,57.9,1.1,35.0,米・精白米（水稲）,66.0,油揚げ,5.0,にんじん・根、皮つき＿生,7.0,板こんにゃく（精粉こんにゃく）,15.0,ごぼう・根＿生,20.0,生しいたけ＿生,5.0,若鶏・むね、皮なし＿生,25.0,こいくちしょうゆ,3.0,みりん風調味料,3.0,昆布だし,50.0,食塩,0.5,清酒・上撰,1.2,さやいんげん・若ざや＿生,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ごはん,赤飯,289,6.1,1.1,63.8,0.5,0.0,赤飯（もち米製品）,150.0,ごま＿いり,1.0,食塩,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# プレバイオティクスのデータの確認
df_prebiotics.head()

,野菜類,豆類,フルーツ,海藻類,きのこ類,その他
0,アスパラガス,大豆,バナナ,わかめ,しいたけ,はちみつ
1,ブロッコリー,レンズ豆,りんご,昆布,えのきたけ,NaN
2,キャベツ,ひよこ豆,ベリー,ひじき,まいたけ,NaN
3,玉ねぎ,インゲン豆,キウイ,のり,えのき,NaN
4,ごぼう,NaN,NaN,NaN,NaN,NaN


# 料理データセットの前処理

In [39]:
# 材料の列の欠損値を" - "で穴埋めする
df_recipes.iloc[:,8::] = df_recipes.iloc[:,8::].fillna('-');
print("処理が完了しました")

処理が完了しました


In [28]:
df_recipes.head()

,料理の種類,料理名,エネルギー[キロカロリー],たんぱく質[グラム],脂質[グラム],炭水化物[グラム],食塩相当量[グラム],野菜量[グラム],材料1＿材料名,材料1＿重量[グラム],材料2＿材料名,材料2＿重量[グラム],材料3＿材料名,材料3＿重量[グラム],材料4＿材料名,材料4＿重量[グラム],材料5＿材料名,材料5＿重量[グラム],材料6＿材料名,材料6＿重量[グラム],材料7＿材料名,材料7＿重量[グラム],材料8＿材料名,材料8＿重量[グラム],材料9＿材料名,材料9＿重量[グラム],材料10＿材料名,材料10＿重量[グラム],材料11＿材料名,材料11＿重量[グラム],材料12＿材料名,材料12＿重量[グラム],材料13＿材料名,材料13＿重量[グラム],材料14＿材料名,材料14＿重量[グラム],材料15＿材料名,材料15＿重量[グラム],材料16＿材料名,材料16＿重量[グラム],材料17＿材料名,材料17＿重量[グラム],材料18＿材料名,材料18＿重量[グラム],材料19＿材料名,材料19＿重量[グラム],材料20＿材料名,材料20＿重量[グラム]
0,ごはん,ごはん（中茶碗1杯）,252,3.8,0.5,55.7,0.0,0.0,めし・精白米（水稲）,150.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
1,ごはん,麦ごはん,248,4.3,0.7,54.0,0.0,0.0,米・精白米（水稲）,60.0,大麦・押麦,10.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
2,ごはん,きのこたっぷり混ぜご飯,307,6.6,2.4,61.4,1.0,23.0,米・精白米（水稲）,70.0,えのきたけ＿生,7.0,しめじ・ぶなしめじ＿生,7.0,まいたけ＿生,7.0,油揚げ,5.0,かつお・昆布だし,90.0,こいくちしょうゆ,6.0,みりん風調味料,9.0,清酒・上撰,7.5,糸みつば・葉＿生,2.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
3,ごはん,五目炊き込みご飯,311,11.4,2.7,57.9,1.1,35.0,米・精白米（水稲）,66.0,油揚げ,5.0,にんじん・根、皮つき＿生,7.0,板こんにゃく（精粉こんにゃく）,15.0,ごぼう・根＿生,20.0,生しいたけ＿生,5.0,若鶏・むね、皮なし＿生,25.0,こいくちしょうゆ,3.0,みりん風調味料,3.0,昆布だし,50.0,食塩,0.5,清酒・上撰,1.2,さやいんげん・若ざや＿生,3.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-
4,ごはん,赤飯,289,6.1,1.1,63.8,0.5,0.0,赤飯（もち米製品）,150.0,ごま＿いり,1.0,食塩,0.5,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-


In [29]:
# 「・」「＿」「（）」以降の単語を削除する関数
def clean_foodname(name):
  """
  IN:鶏卵・全卵＿生
  OUT:鶏卵
  """
  # 「（～）」を削除
  name = re.sub("（[^）]*）", "", name)
  # 「・」「＿」以降も削除
  name = re.sub("[・＿].*", "", name)
  return name

In [30]:
# clean_foodname関数の実行
df_recipes.iloc[:,8::2] = df_recipes.iloc[:,8::2].applymap(clean_foodname)

/tmp/ipython-input-1019100835.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_recipes.iloc[:,8::2] = df_recipes.iloc[:,8::2].applymap(clean_foodname)


In [31]:
# プレバイオティクス一覧を参照し、当てはまればOne-Hot-Encordingする
pre_word = df_prebiotics.stack().tolist() # データフレームの列をリスト化する、「nan」は除外する
# ラベル付け
def check_prebiotics(cell):
  """
  それぞれのレシピにプレバイオティクス一覧の食材が含まれているかどうかを判定する
  OUT:
  含まれている→　1
  含まれていない　→　0
  ※　材料を個別に判断するのではなく、料理全体（1行の中を判断する
  """
  for x in pre_word:
    if x in cell:
      return 1
  return 0

In [32]:
# check_prebiotics関数の実行
df_recipes['プレバイオティクス（全体）'] = df_recipes.iloc[:,8::2].applymap(check_prebiotics).max(axis=1)

/tmp/ipython-input-3022261290.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_recipes['プレバイオティクス（全体）'] = df_recipes.iloc[:,8::2].applymap(check_prebiotics).max(axis=1)


In [33]:
# プロバイオティクス一覧を参照し、当てはまればOne-Hot-Encordingする
pro_word = df_probiotics.values.ravel().tolist() # データフレームの列をリスト化する

In [43]:
# プロバイオティクスのラベル付け
def check_probiotics(cell):
  """
  それぞれのレシピにプロバイオティクス一覧の食材が含まれているかどうかを判定する
  OUT:
  含まれている→　1
  含まれていない　→　0
  ※　材料を個別に判断するのではなく、料理全体（1行の中）を判断する
  """
  for x in pro_word:
    if x in cell:
      return 1
  return 0

In [44]:
# check_prebiotics関数の実行
# 食材の列の範囲（インデックス8～46までの列）
df_recipes['プロバイオティクス（全体）'] = df_recipes.iloc[:,8:46:2].applymap(check_probiotics).max(axis=1)

/tmp/ipython-input-1699891086.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_recipes['プロバイオティクス（全体）'] = df_recipes.iloc[:,8:46:2].applymap(check_probiotics).max(axis=1)


In [45]:
df_recipes.head()

,料理の種類,料理名,エネルギー[キロカロリー],たんぱく質[グラム],脂質[グラム],炭水化物[グラム],食塩相当量[グラム],野菜量[グラム],材料1＿材料名,材料1＿重量[グラム],材料2＿材料名,材料2＿重量[グラム],材料3＿材料名,材料3＿重量[グラム],材料4＿材料名,材料4＿重量[グラム],材料5＿材料名,材料5＿重量[グラム],材料6＿材料名,材料6＿重量[グラム],材料7＿材料名,材料7＿重量[グラム],材料8＿材料名,材料8＿重量[グラム],材料9＿材料名,材料9＿重量[グラム],材料10＿材料名,材料10＿重量[グラム],材料11＿材料名,材料11＿重量[グラム],材料12＿材料名,材料12＿重量[グラム],材料13＿材料名,材料13＿重量[グラム],材料14＿材料名,材料14＿重量[グラム],材料15＿材料名,材料15＿重量[グラム],材料16＿材料名,材料16＿重量[グラム],材料17＿材料名,材料17＿重量[グラム],材料18＿材料名,材料18＿重量[グラム],材料19＿材料名,材料19＿重量[グラム],材料20＿材料名,材料20＿重量[グラム],プレバイオティクス（全体）,プロバイオティクス（全体）,プレプロスコア
0,ごはん,ごはん（中茶碗1杯）,252,3.8,0.5,55.7,0.0,0.0,めし,150.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0,0,0
1,ごはん,麦ごはん,248,4.3,0.7,54.0,0.0,0.0,米,60.0,大麦,10.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0,0,0
2,ごはん,きのこたっぷり混ぜご飯,307,6.6,2.4,61.4,1.0,23.0,米,70.0,えのきたけ,7.0,しめじ,7.0,まいたけ,7.0,油揚げ,5.0,かつお,90.0,こいくちしょうゆ,6.0,みりん風調味料,9.0,清酒,7.5,糸みつば,2.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1,0,15
3,ごはん,五目炊き込みご飯,311,11.4,2.7,57.9,1.1,35.0,米,66.0,油揚げ,5.0,にんじん,7.0,板こんにゃく,15.0,ごぼう,20.0,生しいたけ,5.0,若鶏,25.0,こいくちしょうゆ,3.0,みりん風調味料,3.0,昆布だし,50.0,食塩,0.5,清酒,1.2,さやいんげん,3.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1,0,15
4,ごはん,赤飯,289,6.1,1.1,63.8,0.5,0.0,赤飯,150.0,ごま,1.0,食塩,0.5,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0,0,0


# 腸活スコアの算出

In [46]:
# ①プレバイオティクス・プロバイオティクスの点数
# プレバイオティクス：15 / 30 点　プロバイオティクス：15 / 30点）
# プレプロスコア関数
def prepro_score(row):
  score = 0
  if row['プレバイオティクス（全体）'] == 1: # プレバイオティクスが 1 （食材が含まれている）とき +15点 する
    score += 15
  if row['プロバイオティクス（全体）'] == 1: # プロバイオティクスが 1 （食材が含まれている）とき +15点 する
    score += 15
  return score

# 関数の実行
df_recipes['プレプロスコア']  = df_recipes.apply(prepro_score, axis=1)

In [48]:
df_recipes[df_recipes['プレバイオティクス（全体）'] == 1]

,料理の種類,料理名,エネルギー[キロカロリー],たんぱく質[グラム],脂質[グラム],炭水化物[グラム],食塩相当量[グラム],野菜量[グラム],材料1＿材料名,材料1＿重量[グラム],材料2＿材料名,材料2＿重量[グラム],材料3＿材料名,材料3＿重量[グラム],材料4＿材料名,材料4＿重量[グラム],材料5＿材料名,材料5＿重量[グラム],材料6＿材料名,材料6＿重量[グラム],材料7＿材料名,材料7＿重量[グラム],材料8＿材料名,材料8＿重量[グラム],材料9＿材料名,材料9＿重量[グラム],材料10＿材料名,材料10＿重量[グラム],材料11＿材料名,材料11＿重量[グラム],材料12＿材料名,材料12＿重量[グラム],材料13＿材料名,材料13＿重量[グラム],材料14＿材料名,材料14＿重量[グラム],材料15＿材料名,材料15＿重量[グラム],材料16＿材料名,材料16＿重量[グラム],材料17＿材料名,材料17＿重量[グラム],材料18＿材料名,材料18＿重量[グラム],材料19＿材料名,材料19＿重量[グラム],材料20＿材料名,材料20＿重量[グラム],プレバイオティクス（全体）,プロバイオティクス（全体）,プレプロスコア
2,ごはん,きのこたっぷり混ぜご飯,307,6.6,2.4,61.4,1.0,23.0,米,70.0,えのきたけ,7.0,しめじ,7.0,まいたけ,7.0,油揚げ,5.0,かつお,90.0,こいくちしょうゆ,6.0,みりん風調味料,9.0,清酒,7.5,糸みつば,2.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1,0,15
3,ごはん,五目炊き込みご飯,311,11.4,2.7,57.9,1.1,35.0,米,66.0,油揚げ,5.0,にんじん,7.0,板こんにゃく,15.0,ごぼう,20.0,生しいたけ,5.0,若鶏,25.0,こいくちしょうゆ,3.0,みりん風調味料,3.0,昆布だし,50.0,食塩,0.5,清酒,1.2,さやいんげん,3.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1,0,15
6,ごはん,巻き寿司,402,10.2,4.3,78.2,2.4,73.4,めし,170.0,清酒,3.75,穀物酢,13.0,車糖,4.5,食塩,1.5,乾ししいたけ,2.0,車糖,1.5,みりん風調味料,3.0,こいくちしょうゆ,3.0,鶏卵,25.0,車糖,1.5,食塩,0.2,調合油,1.0,きゅうり,60.0,まだら,3.0,あまのり,2.0,-,-,-,-,-,-,-,-,1,0,15
7,ごはん,ちらし寿司,457,13.7,4.7,86.3,3.2,42.0,めし,200.0,穀物酢,20.0,車糖,4.5,食塩,2.0,にんじん,15.0,食塩,0.2,車糖,1.0,昆布だし,10.0,乾ししいたけ,2.0,かんぴょう,2.0,こいくちしょうゆ,3.0,車糖,1.5,鶏卵,28.0,食塩,0.2,調合油,1.0,くるまえび,20.0,食塩,0.2,さやいんげん,5.0,-,-,-,-,1,0,15
8,ごはん,親子丼,665,28.5,8.9,110.7,2.1,65.0,めし,260.0,若鶏,75.0,清酒,3.75,たまねぎ,60.0,かつおだし,25.0,こいくちしょうゆ,12.0,みりん風調味料,9.0,車糖,2.25,鶏卵,50.0,切りみつば,5.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1,0,15
9,ごはん,カツ丼,868,30.5,31.1,107.3,2.8,20.0,めし,260.0,豚,80.0,食塩,1.0,薄力粉,4.0,鶏卵,4.0,パン粉,4.0,調合油,9.0,たまねぎ,20.0,こいくちしょうゆ,10.0,みりん,5.0,鶏卵,50.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1,0,15
10,ごはん,牛丼,873,15.7,35.8,112.8,2.2,30.0,めし,260.0,和牛,70.0,こんにゃく,30.0,たまねぎ,30.0,車糖,7.0,こいくちしょうゆ,15.0,みりん,10.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1,0,15
12,ごはん,中華丼,743,31.1,20.3,101.8,2.8,40.0,めし,260.0,いか,20.0,しばえび,20.0,豚,30.0,うずら卵,15.0,はくさい,20.0,たけのこ,10.0,にんじん,5.0,根深ねぎ,5.0,調合油,6.0,こいくちしょうゆ,5.0,食塩,1.0,こしょう,0.02,顆粒風味調味料,1.0,じゃがいもでん粉,3.0,-,-,-,-,-,-,-,-,-,-,1,0,15
13,ごはん,カレーライス（ビーフ）,882,20.1,31.1,124.2,3.2,90.0,めし,260.0,和牛,60.0,じゃがいも,60.0,たまねぎ,50.0,にんじん,40.0,調合油,10.0,食塩,1.0,こしょう,0.02,カレールウ,20.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1,0,15
14,ごはん,オムライス,596,20.2,16.4,86.8,2.3,52.0,めし,200.0,若鶏,30.0,たまねぎ,40.0,青ピーマン,10.0,調合油,5.0,食塩,1.0,トマト加工品,30.0,鶏卵,60.0,有塩バター,5.0,パセリ,2.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,1,0,15
